In [1]:
%load_ext autoreload
%autoreload 2

In [26]:
from rlmep.envs import ReactorEnv
from rlmep.agents.reinforce import ReinforceAgent, PolicyNetwork
import torch

In [27]:
from torch.distributions import Normal, HalfNormal

mu = torch.tensor([0.1, 2.0])
sigma = torch.tensor([0.1, 2.0])

distribution = HalfNormal(mu, sigma, validate_args=False)

action = distribution.sample()

logp = distribution.log_prob(action)

logp = logp.sum()

TypeError: HalfNormal.__init__() got multiple values for argument 'validate_args'

In [18]:
env = ReactorEnv()
policy = PolicyNetwork(4, 2)

In [11]:
action, logp = policy.sample_action(torch.tensor([0.0, 0.0, 0.0, 0.0], dtype=torch.float32))

print("Action:", action)
print("Log probability:", logp)

RuntimeError: Boolean value of Tensor with more than one value is ambiguous

In [3]:
agent = ReinforceAgent(policy, env)
agent.play_episode()

TypeError: cannot unpack non-iterable int object

In [ ]:
state, _ = env.reset()

state = torch.tensor(state, dtype=torch.float32)

action, log_p = policy.sample_action(state)